## Shor's Algorithm

Shor's algorithm is used to find prime factors of an integer. On a quantum computer, Shor's algorithm runs in polynomial time and is almost exponentially faster than the most efficient known classical factoring algorithm. The efficiency of Shor's algorithm is due to the efficiency of the Quantum Fourier transform, Quantum Phase estimation and modular exponentiation by repeated squarings. In this notebook, you implement the Shor's algorithm in code using the Amazon Braket SDK and run a simple example of factoring 15.

## Implementation of Shor's algorithm
This example provides an implementation of the Shor's algorithm using the Amazon Braket SDK. This has been installed in the preparation steps.

### General steps
* Copy the code into `test.py`
* Any references/imports are of modules that have been installed prior to the start of this lab
* Run the program: `python test.py` from a terminal to see the results

In [1]:
from braket.devices import LocalSimulator
from braket.aws import AwsDevice
from braket.experimental.algorithms.shors.shors import (
    shors_algorithm,
    run_shors_algorithm,
    get_factors_from_results
)

def main():
    # Main function

    #  #  # Place the other code here #  #  #

    exit


if __name__ == '__main__':
    main()


## Prepare inputs for Shor's Algorithm

### Quantum computer's circuit
To perform a computation of Shor's algorithm a quantum circuit is required. A quantum circuit is a model for quantum computation, similar to classical circuits, in which a computation is a sequence of quantum gates, measurements, initializations of qubits to known values, and possibly other actions.

We create the circuit by calling the function `shors_algorithm`

In [2]:
def main():

    # ...
    
    N = 15 # Integer to factor (currently 15, 21, 35 work)
    a = 7  # Any integer that satisfies 1 < a < N and gcd(a, N) = 1.

    shors_circuit = shors_algorithm(N, a)

## Run on a local simulator

Now that we have a circuit we can run a local Quantum simulator

In [ ]:

def main():

    # ...

    local_simulator = LocalSimulator()

    output = run_shors_algorithm(shors_circuit, local_simulator)

    guessed_factors = get_factors_from_results(output, N, a)

    print(guessed_factors)